<a href="https://colab.research.google.com/github/Bryan-Az/RL-DQN-Gym/blob/main/DQN_projects/%5BFinal_Evaluation_and_Continued_Training%5D_Deep_Reinforcement_Learning_via_Neural_Network_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Deep Q-Learning Network (DQN) Reinforcement Learning Agent for Blackjack**

## Imports and Installs

In [4]:
%%capture
# capture line is to hide the output
# Install required packages
!pip install gymnasium torch numpy matplotlib

In [1]:
import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from collections import deque
import random

## **DQN Agent with Replay Memory Implementation**

In [15]:
import random
from collections import namedtuple, deque

# Define a named tuple to store experiences
Experience = namedtuple('Experience', ('state', 'action', 'reward', 'next_state', 'done'))

class ReplayMemory:
    def __init__(self, capacity=10000):
        self.memory = deque(maxlen=capacity)
        self.capacity = capacity

    def push(self, state, action, reward, next_state, done):
        """Save an experience to memory"""
        experience = Experience(state, action, reward, next_state, done)
        self.memory.append(experience)

    def sample(self, batch_size):
        """Randomly sample a batch of experiences from memory"""
        if batch_size > len(self.memory):
            batch_size = len(self.memory)
        experiences = random.sample(self.memory, batch_size)

        # Convert to separate arrays
        states = torch.FloatTensor([exp.state for exp in experiences])
        actions = torch.LongTensor([exp.action for exp in experiences])
        rewards = torch.FloatTensor([exp.reward for exp in experiences])
        next_states = torch.FloatTensor([exp.next_state for exp in experiences])
        dones = torch.FloatTensor([exp.done for exp in experiences])

        return states, actions, rewards, next_states, dones

    def __len__(self):
        return len(self.memory)

class ImprovedDQNAgent:
    def __init__(self, input_dim=3, learning_rate=5e-4, gamma=0.99, epsilon=1.0):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.fitness = [] # used to store a series of 'avg reward' during evaluation

        # Larger network
        self.policy_net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 2)
        ).to(self.device)

        self.target_net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 2)
        ).to(self.device)

        self.target_net.load_state_dict(self.policy_net.state_dict())

        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=learning_rate)
        self.memory = ReplayMemory(capacity=20000)

        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = 0.05
        self.epsilon_decay = 0.99997
        self.batch_size = 128
        self.target_update = 10
        self.episode_count = 0

    def select_action(self, state):
        """Select action using epsilon-greedy policy"""
        if random.random() < self.epsilon:
            return random.randint(0, 1)

        with torch.no_grad():
            state = torch.FloatTensor(state).unsqueeze(0).to(self.device)
            q_values = self.policy_net(state)
            return q_values.argmax().item()

    def update_epsilon(self):
        """Decay epsilon value"""
        self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

    def store_transition(self, state, action, reward, next_state, done):
        """Store transition in replay memory"""
        self.memory.push(state, action, reward, next_state, done)

    def train_step(self):
        """Perform one training step"""
        if len(self.memory) < self.batch_size:
            return

        states, actions, rewards, next_states, dones = self.memory.sample(self.batch_size)
        states = states.to(self.device)
        actions = actions.to(self.device)
        rewards = rewards.to(self.device)
        next_states = next_states.to(self.device)
        dones = dones.to(self.device)

        # Double DQN implementation
        with torch.no_grad():
            next_actions = self.policy_net(next_states).argmax(1)
            next_q_values = self.target_net(next_states).gather(1, next_actions.unsqueeze(1)).squeeze(1)
            target_q_values = rewards + (1 - dones.float()) * self.gamma * next_q_values

        current_q_values = self.policy_net(states).gather(1, actions.unsqueeze(1)).squeeze(1)

        # Huber loss for better stability
        loss = nn.SmoothL1Loss()(current_q_values, target_q_values)

        self.optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.policy_net.parameters(), 1.0)
        self.optimizer.step()

        return loss.item()

    def update_target_network(self):
        """Update target network parameters"""
        if self.episode_count % self.target_update == 0:
            self.target_net.load_state_dict(self.policy_net.state_dict())
        self.episode_count += 1

In [18]:
# Initialize the improved agent then loading the state dict from the .pth file
DQN_Agent = ImprovedDQNAgent()
DQN_Agent.policy_net.load_state_dict(torch.load('/content/Blackjack_DQN_500000_episodes.pth')['policy_net_state_dict'])

<ipython-input-18-6ea958eb1ffb>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  DQN_Agent.policy_net.load_state_dict(torch.load('/content/Blackjack_DQN_500000_episodes.pth'

<All keys matched successfully>

## Evaluation

### Evaluation Metrics of the Trained Model on a New Environment

In [19]:
test_env = gym.make("Blackjack-v1", render_mode='rgb_array')

In [20]:
# Evaluate the trained agent
def evaluate_agent_detailed(test_env, trained_agent, n_episodes=200):
    wins = 0
    draws = 0
    losses = 0
    total_rewards = []
    player_sums = []
    dealer_sums = []

    for episode in range(n_episodes):
        state, _ = test_env.reset()
        done = False
        episode_reward = 0

        while not done:
            # Use greedy policy (no exploration)
            with torch.no_grad():
                state_tensor = torch.FloatTensor(state).unsqueeze(0)
                action = trained_agent.policy_net(state_tensor).argmax().item()

            state, reward, done, truncated, _ = test_env.step(action)
            episode_reward += reward

        total_rewards.append(episode_reward)
        if reward > 0:
            wins += 1
        elif reward == 0:
            draws += 1
        else:
            losses += 1

        player_sums.append(state[0])  # Final player sum

    print("\nDetailed Evaluation Results:")
    print(f"Number of Episodes: {n_episodes}")
    print(f"Win Rate: {wins/n_episodes*100:.1f}%  ({wins}/{n_episodes})")
    print(f"Draw Rate: {draws/n_episodes*100:.1f}%  ({draws}/{n_episodes})")
    print(f"Loss Rate: {losses/n_episodes*100:.1f}%  ({losses}/{n_episodes})")
    print(f"Average Reward: {np.mean(total_rewards):.3f}")
    print(f"Average Final Player Sum: {np.mean(player_sums):.1f}")

    return total_rewards, player_sums

# Evaluate the trained agent
print("\nEvaluating trained agent...")
eval_rewards, player_sums = evaluate_agent_detailed(test_env, DQN_Agent, n_episodes=200)


Evaluating trained agent...

Detailed Evaluation Results:
Number of Episodes: 200
Win Rate: 38.5%  (77/200)
Draw Rate: 14.0%  (28/200)
Loss Rate: 47.5%  (95/200)
Average Reward: -0.090
Average Final Player Sum: 19.6


### Visualizing the Agent's Learning with Gymnasium's RGB Environment

In [24]:
# Uses the Gym Monitor wrapper to evalaute the agent and record video
# only one video will be saved
# video of the final episode with the episode trigger
test_env = gym.wrappers.RecordVideo(
    test_env, "./gym_monitor_output", episode_trigger=lambda x: x == 0)

evaluate_agent_detailed(test_env, DQN_Agent)

test_env.close()

/usr/local/lib/python3.10/dist-packages/gymnasium/wrappers/rendering.py:283: UserWarning: WARN: Overwriting existing videos at /content/gym_monitor_output folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(



Detailed Evaluation Results:
Number of Episodes: 200
Win Rate: 35.0%  (70/200)
Draw Rate: 9.5%  (19/200)
Loss Rate: 55.5%  (111/200)
Average Reward: -0.205
Average Final Player Sum: 19.5


In [25]:
# play a video using a path to the video
from IPython.display import Video
from base64 import b64encode

def show_video(video_path):
    video_file = Video(video_path, embed=True)
    display(video_file)


In [26]:
# visualizing the rl-video-episode-0.mp4 in the gym_monitor_output
show_video("./gym_monitor_output/rl-video-episode-0.mp4")

## Strategy Analysis and Interactive Human Feedback Loop with A.I Recommendation

### Strategy Analysis

In [29]:
def analyze_strategy(agent):
    # Common situations in blackjack
    test_states = [
        (16, 10, 0),  # Hard 16 vs dealer 10
        (12, 6, 0),   # Hard 12 vs dealer 6
        (18, 9, 0),   # Hard 18 vs dealer 9
        (11, 10, 0),  # Hard 11 vs dealer 10
        (15, 7, 0),   # Hard 15 vs dealer 7
    ]

    print("\nStrategy Analysis:")
    print("Player Sum | Dealer Card | Action")
    print("-" * 35)

    for player_sum, dealer_card, usable_ace in test_states:
        state = np.array([player_sum, dealer_card, usable_ace])
        with torch.no_grad():
            state_tensor = torch.FloatTensor(state).unsqueeze(0)
            action = agent.policy_net(state_tensor).argmax().item()
            action_name = "Hit" if action == 1 else "Stand"
            print(f"{player_sum:^10} | {dealer_card:^11} | {action_name:^6}")

# Run strategy analysis
analyze_strategy(DQN_Agent)


Strategy Analysis:
Player Sum | Dealer Card | Action
-----------------------------------
    16     |     10      |  Hit  
    12     |      6      | Stand 
    18     |      9      | Stand 
    11     |     10      |  Hit  
    15     |      7      |  Hit  


### Interactive Human Feedback Loop

In [31]:
def print_cards(cards, hidden=False):
    if hidden:
        return f"[{cards[0]}, ?]"
    return str(cards)

def get_card_value(card):
    if card == 1:  # Ace
        return 11
    return min(card, 10)

def calculate_hand_value(cards):
    value = sum(get_card_value(card) for card in cards)
    num_aces = cards.count(1)

    # Adjust for aces
    while value > 21 and num_aces:
        value -= 10
        num_aces -= 1

    return value

def play_interactive_blackjack_with_learning(agent):
    print("\nWelcome to Interactive Learning Blackjack!")
    print("The AI will learn from your games.")

    game_memory = []  # Store game experiences
    stats = {'games': 0, 'wins': 0, 'losses': 0, 'draws': 0}

    while True:
        player_cards = []
        dealer_cards = []
        deck = list(range(1, 14)) * 4
        random.shuffle(deck)

        # Initial deal
        player_cards.extend([deck.pop(), deck.pop()])
        dealer_cards.extend([deck.pop(), deck.pop()])

        game_states = []  # Store states for this game

        while True:
            print("\nDealer shows:", print_cards(dealer_cards, hidden=True))
            print("Your cards:", print_cards(player_cards))
            player_value = calculate_hand_value(player_cards)
            print(f"Your total: {player_value}")

            if player_value > 21:
                print("Bust! You lose.")
                stats['losses'] += 1
                break

            # Current state
            current_state = np.array([player_value, get_card_value(dealer_cards[0]), 1 in player_cards])

            # Get AI recommendation
            with torch.no_grad():
                state_tensor = torch.FloatTensor(current_state).unsqueeze(0)
                q_values = agent.policy_net(state_tensor)
                ai_action = q_values.argmax().item()
                confidence = torch.softmax(q_values, dim=1)[0]
                ai_recommendation = "Hit" if ai_action == 1 else "Stand"
                print(f"\nAI Recommends: {ai_recommendation} (Confidence: {confidence[ai_action]:.2f})")

            action = input("\nYour action (H/S): ").upper()
            while action not in ['H', 'S']:
                action = input("Invalid input. Please enter H or S: ").upper()

            # Store state and action
            game_states.append((
                current_state,
                1 if action == 'H' else 0,
                player_value
            ))

            if action == 'H':
                player_cards.append(deck.pop())
            else:
                break

        # Game ended, calculate final reward
        final_player_value = calculate_hand_value(player_cards)
        dealer_value = play_dealer_hand(dealer_cards, deck)

        if final_player_value <= 21:
            print("\nFinal hands:")
            print(f"Dealer: {print_cards(dealer_cards)} (Total: {dealer_value})")
            print(f"Player: {print_cards(player_cards)} (Total: {final_player_value})")

            if dealer_value > 21:
                print("Dealer busts! You win!")
                reward = 1.0
                stats['wins'] += 1
            elif dealer_value > final_player_value:
                print("Dealer wins!")
                reward = -1.0
                stats['losses'] += 1
            elif dealer_value < final_player_value:
                print("You win!")
                reward = 1.0
                stats['wins'] += 1
            else:
                print("Push (tie)!")
                reward = 0.0
                stats['draws'] += 1
        else:
            reward = -1.0

        # Store experiences for learning
        for state, action, value in game_states:
            agent.memory.push(state, action, reward, state, True)
            # Train on a batch
            if len(agent.memory) >= agent.batch_size:
                loss = agent.train_step()
                if loss:
                    print(f"Training loss: {loss:.4f}")

        stats['games'] += 1
        print("\nCurrent Stats:")
        print(f"Games Played: {stats['games']}")
        print(f"Win Rate: {stats['wins']/stats['games']*100:.1f}%")

        play_again = input("\nPlay again? (Y/N): ").upper()
        if play_again != 'Y':
            break

    print("\nFinal Stats:")
    print(f"Games Played: {stats['games']}")
    print(f"Wins: {stats['wins']}")
    print(f"Losses: {stats['losses']}")
    print(f"Draws: {stats['draws']}")
    print(f"Win Rate: {stats['wins']/stats['games']*100:.1f}%")

    # Save the improved model
    torch.save({
        'policy_net_state_dict': agent.policy_net.state_dict(),
        'optimizer_state_dict': agent.optimizer.state_dict(),
        'epsilon': agent.epsilon,
    }, 'blackjack_dqn_improved.pth')
    print("\nImproved model saved!")

# Helper function for dealer's turn
def play_dealer_hand(dealer_cards, deck):
    dealer_value = calculate_hand_value(dealer_cards)
    while dealer_value < 17:
        dealer_cards.append(deck.pop())
        dealer_value = calculate_hand_value(dealer_cards)
        print(f"Dealer hits: {print_cards(dealer_cards)} (Total: {dealer_value})")
    return dealer_value

# Start the learning interactive game
print("\nStarting interactive learning blackjack game...")
play_interactive_blackjack_with_learning(DQN_Agent)


Starting interactive learning blackjack game...

Welcome to Interactive Learning Blackjack!
The AI will learn from your games.

Dealer shows: [6, ?]
Your cards: [1, 7]
Your total: 18

AI Recommends: Stand (Confidence: 0.52)

Your action (H/S): S
Dealer hits: [6, 12, 2] (Total: 18)

Final hands:
Dealer: [6, 12, 2] (Total: 18)
Player: [1, 7] (Total: 18)
Push (tie)!

Current Stats:
Games Played: 1
Win Rate: 0.0%

Play again? (Y/N): Y

Dealer shows: [4, ?]
Your cards: [12, 8]
Your total: 18

AI Recommends: Stand (Confidence: 0.71)

Your action (H/S): S
Dealer hits: [4, 1, 6] (Total: 21)

Final hands:
Dealer: [4, 1, 6] (Total: 21)
Player: [12, 8] (Total: 18)
Dealer wins!

Current Stats:
Games Played: 2
Win Rate: 0.0%

Play again? (Y/N): N

Final Stats:
Games Played: 2
Wins: 0
Losses: 1
Draws: 1
Win Rate: 0.0%

Improved model saved!
